In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm

import path
import groupXY_functions as util

In [ ]:
# Read in ground truth for the 2000 picture dataset
df = pd.read_csv(path.ground_truth_full)

In [ ]:
def create_training_data(width, height):
    training_data = []

    for _, row in tqdm(df.iterrows()):
        id_ = row['image_id']
        # Load img and mask
        img_arr = cv2.imread(f'{path.images_full}{id_}.jpg')
        mask_arr = cv2.imread(f'{path.segmentation_full}{id_}_segmentation.png', 0)

        # Rescale image and mask
        img_arr, mask_arr = util.scale_image_res(img_arr, mask_arr, dim = (width, height))

        # Mask lesion from image
        masked_img = cv2.bitwise_and(img_arr, img_arr, mask = mask_arr)

        # Append masked image and corrosponding binary classification of melanoma to training data
        training_data.append([masked_img, row['melanoma']])

    return training_data

In [ ]:
# Set dimensions
rescale_width = 350
rescale_height = 260

In [ ]:
training_dataset = create_training_data(rescale_width, rescale_height)

In [ ]:
# Shuffle training data in case theres any patterns in the order of the data
from random import shuffle

shuffle(training_dataset)

In [ ]:
X = []
y = []

# Split features and labels into X and y from training dataset
for features, label in training_dataset:
    X.append(features)
    y.append(label)

# Convert to numpy arrays and reshape data into accepted format of the model
X = np.array(X).reshape(-1, rescale_width, rescale_height, 3)
y = np.array(y)

In [ ]:
# Saving the dataset into pickle files
import pickle

pickle_out = open("X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()